In [2]:
import os
import os.path as path
import re
from enum import Enum

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
import matplotlib.patheffects as PathEffects
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm
# tqdm.pandas(desc='Processing Dataframe', leave=False, position=1)
import tafra
import ptitprince as pt

mpl.style.use('seaborn')
# %matplotlib notebook
# xkcd-colors: https://matplotlib.org/stable/_images/sphx_glr_colors_003.png

In [3]:
participant_survey_file = './participant_survey.csv'
column_mapping = {
    'Subject Information [Subject ID]' : 'SubjectId',
    'Do you feel like you had enough training time with the simulated phosphene vision and the different conditions?  Use the comment box to add an explanation / details. '
    'Order in which conditions occured [Rank 1]' : "1stCondition",
    'Order in which conditions occured [Rank 2]' : "2ndCondition",
    'Order in which conditions occured [Rank 3]' : "3rdCondition",
    'Rate the conditions in regard to how well you could navigate in each of them from 1 to 5. A lower number indicates you had a harder time navigating in it.  [1st trial / condition of this block]' : "1stNavRating",
    'Rate the conditions in regard to how well you could navigate in each of them from 1 to 5. A lower number indicates you had a harder time navigating in it.  [2nd trial / condition of this block]' : "2ndNavRating",
    'Rate the conditions in regard to how well you could navigate in each of them from 1 to 5. A lower number indicates you had a harder time navigating in it.  [3rd trial / condition of this block]' : "3rdNavRating",
    'Which condition do you think you performed best in?' : "BestCondition",
    'Select all conditions that you experienced nausea or motion sickness in. [1st trial / condition of this block]' : "1stNausea",
    'Select all conditions that you experienced nausea or motion sickness in. [2nd trial / condition of this block]' : "2ndNausea",
    'Select all conditions that you experienced nausea or motion sickness in. [3rd trial / condition of this block]' : "3rdNausea",
    'Rank the conditions in order of how tiring they were, with the least tiring at the top and most tiring at the bottom. [Rank 1]' : "1stTiring",
    'Rank the conditions in order of how tiring they were, with the least tiring at the top and most tiring at the bottom. [Rank 2]' : "2ndTiring",
    'Rank the conditions in order of how tiring they were, with the least tiring at the top and most tiring at the bottom. [Rank 3]' : "3rdTiring",
    'Rank the conditions in order of comfort, with the most comfortable at the top and least comfortable at the bottom. [Rank 1]' : "1stComfort",
    'Rank the conditions in order of comfort, with the most comfortable at the top and least comfortable at the bottom. [Rank 2]' : "2ndComfort",
    'Rank the conditions in order of comfort, with the most comfortable at the top and least comfortable at the bottom. [Rank 3]' : "3rdComfort",
    'Mark how much you feel the statement applies to you right now. [I feel tired / distracted]' : "Tired_Distracted",
    'Mark how much you feel the statement applies to you right now. [I can concentrate]' : "Concentration",
    'Order the conditions in the order they appeared in this block, with the first condition at the top and the last condition at the bottom: [Rank 1]' : "1stConditionSubj",
    'Order the conditions in the order they appeared in this block, with the first condition at the top and the last condition at the bottom: [Rank 2]' : "2ndConditionSubj",
    'Order the conditions in the order they appeared in this block, with the first condition at the top and the last condition at the bottom: [Rank 3]' : "3rdConditionSubj",
    'Anything else you would like to note about this block? (e.g. technical issues, notable pysical or emotional reactions, anything you would have liked to add to the previous questions, ...)' : "Comments"
}

In [4]:
df = pd.read_csv(participant_survey_file, sep='\t')
# df = df.rename(columns=column_mapping)
# df.set_index(['SubjectId', 'Block'], inplace=True)
# df = df.replace(to_replace = {"Condition A (eye movement ignored)" : "Gaze Ignored",
#                     "Condition B (locked to your focus point)" : "Gaze Locked",
#                     "Condition C (eye movement to scan the scene)" : "Gaze Asissted" })
df.head()
[print(f"'{c}' : {c.split('[')[-1][:-1]}") for c in df.columns]

'Subject Information [Subject ID]' : Subject ID
'Do you feel like you had enough training time with the simulated phosphene vision and the different conditions?  Use the comment box to add an explanation / details. ' : Do you feel like you had enough training time with the simulated phosphene vision and the different conditions?  Use the comment box to add an explanation / details.
'Do you feel like you had enough training time with the simulated phosphene vision and the different conditions?  Use the comment box to add an explanation / details.  [Comment]' : Comment
'Did you feel like you were still getting better throughout the trials in interpreting the simulated phosphene vision?  Use the comment box to add an explanation / details. ' : Did you feel like you were still getting better throughout the trials in interpreting the simulated phosphene vision?  Use the comment box to add an explanation / details.
'Did you feel like you were still getting better throughout the trials in int

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]